<a href="https://colab.research.google.com/github/Samundra2005/MeroCode/blob/master/Grid_Map_Game_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import time
import math
import random

# Global cache for AI moves. Must be cleared before each new game.
memoization_cache = {}

# --------------------------------------------------------------------------
# --- AI BRAIN (CORE SOLVER LOGIC) ---
# --------------------------------------------------------------------------

def evaluate_final_state(grid, p1_troops, p2_troops, initiative_winner_known):
    """Calculates the final score of a terminal board state (+1 for P1 win, -1 for P2 win) by simulating the attack phase."""
    final_grid = [row[:] for row in grid]

    # Determine Initiative Winner (Fair Tie-Breaker Rule)
    if initiative_winner_known:
        first_attacker = initiative_winner_known
    else:
        first_attacker = 'P2'  # Give tie-breaker to P2 for fairness

    second_attacker = 'P2' if first_attacker == 'P1' else 'P1'

    # Simulate the two attack phases
    final_grid = run_simulated_attack(final_grid, first_attacker)
    final_grid = run_simulated_attack(final_grid, second_attacker)

    # Calculate final winner
    p1_cells, p2_cells, final_p1_troops, final_p2_troops = 0, 0, 0, 0
    for row in final_grid:
        for cell in row:
            if cell:
                player, troops = cell
                if player == 'P1':
                    p1_cells += 1
                    final_p1_troops += troops
                else:
                    p2_cells += 1
                    final_p2_troops += troops

    if p1_cells > p2_cells: return 1
    if p2_cells > p1_cells: return -1
    if final_p1_troops > final_p2_troops: return 1
    if final_p2_troops > final_p1_troops: return -1
    return 1 if first_attacker == 'P1' else -1

def run_simulated_attack(grid, attacking_player):
    """Internal attack simulation for the AI. No print statements."""
    if not attacking_player: return grid
    defending_player = 'P2' if attacking_player == 'P1' else 'P1'
    rows, cols = len(grid), len(grid[0])
    cells_to_remove = []

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] and grid[r][c][0] == defending_player:
                defender_troops = grid[r][c][1]
                total_attacker_strength = 0
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if dr == 0 and dc == 0: continue
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] and grid[nr][nc][0] == attacking_player:
                            is_3x3_grid = (rows == 3 and cols == 3)
                            attacker_is_center = (nr == 1 and nc == 1)
                            is_diagonal_attack = (dr != 0 and dc != 0)
                            if is_3x3_grid and attacker_is_center and is_diagonal_attack:
                                continue
                            total_attacker_strength += grid[nr][nc][1]
                if total_attacker_strength > defender_troops:
                    cells_to_remove.append((r, c))

    if not cells_to_remove: return grid
    new_grid = [row[:] for row in grid]
    for r_rem, c_rem in cells_to_remove:
        new_grid[r_rem][c_rem] = None
    return new_grid

def get_possible_moves(grid, turn_player, p1_troops, p2_troops):
    """Generates all valid moves for the current player."""
    moves = []
    current_troops = p1_troops if turn_player == 'P1' else p2_troops
    if current_troops == 0: return []
    troops_to_place_options = {1}
    if current_troops > 1:
        troops_to_place_options.add(current_troops)
        if current_troops <= 6:
            for i in range(2, current_troops): troops_to_place_options.add(i)
        else:
            troops_to_place_options.add(current_troops // 2)

    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] is None:
                for k in troops_to_place_options:
                    if k <= current_troops:
                        moves.append({'k': k, 'pos': (r, c)})
    return moves

def find_best_ai_move(grid, p1_troops, p2_troops, turn_player, alpha, beta, initiative_winner, depth, max_depth):
    """Core AI logic. Uses alpha-beta minimax search to find the optimal move within a given search depth. Returns (score, move)."""
    board_full = all(cell is not None for row in grid for cell in row)
    if (p1_troops == 0 and p2_troops == 0) or board_full or depth == max_depth:
        return evaluate_final_state(grid, p1_troops, p2_troops, initiative_winner), None

    canonical_key = (tuple(map(tuple, grid)), p1_troops, p2_troops, turn_player, depth)
    if canonical_key in memoization_cache:
        return memoization_cache[canonical_key]

    possible_moves = get_possible_moves(grid, turn_player, p1_troops, p2_troops)
    possible_moves.sort(key=lambda move: move['k'])

    if not possible_moves:
        score, _ = find_best_ai_move(grid, p1_troops, p2_troops, 'P2' if turn_player == 'P1' else 'P1', alpha, beta, initiative_winner, depth + 1, max_depth)
        return score, None

    best_move_for_this_state = possible_moves[0]
    is_maximizing_player = (turn_player == 'P1')
    best_value = -math.inf if is_maximizing_player else math.inf

    # Store all moves and their resulting scores
    scored_moves = []

    for move in possible_moves:
        r, c = move['pos']
        k = move['k']
        grid[r][c] = (turn_player, k)

        new_p1_troops, new_p2_troops = p1_troops, p2_troops
        new_initiative_winner = initiative_winner

        if turn_player == 'P1':
            new_p1_troops -= k
            if new_p1_troops == 0 and not initiative_winner: new_initiative_winner = 'P1'
        else:
            new_p2_troops -= k
            if new_p2_troops == 0 and not initiative_winner: new_initiative_winner = 'P2'

        value, _ = find_best_ai_move(grid, new_p1_troops, new_p2_troops, 'P2' if turn_player == 'P1' else 'P1', alpha, beta, new_initiative_winner, depth + 1, max_depth)
        scored_moves.append({'move': move, 'score': value})
        grid[r][c] = None # Backtrack

        if is_maximizing_player:
            if value > best_value:
                best_value = value
                best_move_for_this_state = move
            alpha = max(alpha, best_value)
        else:
            if value < best_value:
                best_value = value
                best_move_for_this_state = move
            beta = min(beta, best_value)

        if beta <= alpha: break

    # AI Imperfection Logic: occasionally pick a good, but not perfect, move.
    if max_depth < float('inf'):
        if is_maximizing_player:
            # Sort moves from best (highest score) to worst
            scored_moves.sort(key=lambda x: x['score'], reverse=True)
        else:
            # Sort moves from best (lowest score) to worst
            scored_moves.sort(key=lambda x: x['score'])

        # On Easy (max_depth=2), 25% chance to pick the 2nd best move.
        # On Challenging (max_depth=4), 15% chance to pick the 2nd best move.
        if len(scored_moves) > 1:
            chance_for_mistake = 0.25 if max_depth == 2 else 0.15
            if random.random() < chance_for_mistake:
                best_move_for_this_state = scored_moves[1]['move']

    memoization_cache[canonical_key] = (best_value, best_move_for_this_state)
    return best_value, best_move_for_this_state

# --------------------------------------------------------------------------
# --- GAME SIMULATOR & INTERFACE ---
# --------------------------------------------------------------------------

def display_grid(grid):
    """Displays the game grid in a readable format."""
    header = "     " + " ".join([f"{i:^3}" for i in range(len(grid[0]))])
    print(header)
    print("   " + "-" * (len(grid[0]) * 4 + 1))
    for i, row in enumerate(grid):
        row_str = f"{i:^3} |"
        for cell in row:
            if cell is None: row_str += " .  "
            else:
                player, troops = cell
                row_str += f" \033[94m{player}\033[0m " if player == 'P1' else f" \033[91m{player}\033[0m "
        row_str += "|"
        print(row_str)
        troop_str = "     "
        for cell in row:
            troop_str += f" {cell[1]:<2} " if cell else "    "
        print(troop_str)
    print("   " + "-" * (len(grid[0]) * 4 + 1))

def run_human_vs_ai_game():
    """Drives the complete, interactive simulation of the Aggression game against an AI."""
    print("--- Aggression: Human vs. AI ---")

    # --- Setup Phase ---
    try:
        rows = int(input("Enter grid rows (m, e.g., 3): "))
        cols = int(input("Enter grid columns (n, e.g., 3): "))

        print("\nChoose AI Difficulty:")
        print("1. Easy (A casual opponent that sometimes makes mistakes.)")
        print("2. Challenging (A strategic AI that rarely makes mistakes.)")
        print("3. Complex (A master strategist that plays perfectly.)")
        difficulty_choice = input("Select difficulty (1-3): ")

        max_depth = 2 # Default to Easy
        if difficulty_choice == '2': max_depth = 4
        elif difficulty_choice == '3': max_depth = float('inf')

        human_player = input("Do you want to play as P1 (first move) or P2? ").upper()
        while human_player not in ['P1', 'P2']:
            human_player = input("Invalid choice. Please enter P1 or P2: ").upper()

        ai_player = 'P2' if human_player == 'P1' else 'P1'

        total_troops = rows * cols
        p1_troops, p2_troops = total_troops, total_troops
        turn_number = 1
        grid = [[None for _ in range(cols)] for _ in range(rows)]
        initiative_winner = None
        global memoization_cache
        memoization_cache.clear()

        print("-" * 40)
        print(f"Grid: {rows}x{cols}. Human: {human_player}, AI: {ai_player}.")
        print("-" * 40)

        # --- Main Game Loop ---
        while True:
            display_grid(grid)
            is_p1_turn = (turn_number % 2 != 0)
            current_player = 'P1' if is_p1_turn else 'P2'
            troops_left = p1_troops if is_p1_turn else p2_troops

            if troops_left == 0:
                print(f"\n--- {current_player} has no troops remaining. Turn skipped. ---")
                turn_number += 1
                time.sleep(1)
                if p1_troops == 0 and p2_troops == 0: break
                continue

            print(f"\n--- Turn {turn_number} ({current_player}) ---")
            print(f"Troops Remaining -> P1: {p1_troops} | P2: {p2_troops}")

            if current_player == human_player:
                # Human's Turn
                k = int(input(f"How many troops to place (1 to {troops_left})? "))
                r, c = map(int, input(f"Place at row,col (e.g., '0,0'): ").split(','))
                if not (0 <= r < rows and 0 <= c < cols and grid[r][c] is None and 1 <= k <= troops_left):
                    print("\033[91mInvalid move. Please check coordinates and troop count.\033[0m")
                    continue
                grid[r][c] = (human_player, k)
            else:
                # AI's Turn
                print(f"AI ({ai_player}) is calculating its move...")
                _, best_move = find_best_ai_move(grid, p1_troops, p2_troops, ai_player, -math.inf, math.inf, initiative_winner, 0, max_depth)

                r, c = best_move['pos']
                k = best_move['k']
                print(f"AI places {k} troops at ({r}, {c}).")
                grid[r][c] = (ai_player, k)

            # Update State
            if is_p1_turn: p1_troops -= k
            else: p2_troops -= k

            if not initiative_winner:
                if p1_troops == 0: initiative_winner = 'P1'
                elif p2_troops == 0: initiative_winner = 'P2'

            board_full = all(cell is not None for row in grid for cell in row)
            if (p1_troops == 0 and p2_troops == 0) or board_full:
                break
            turn_number += 1

        # --- End Game Logic ---
        print("\n" + "="*40)
        print("        PLACEMENT COMPLETE - FINAL BATTLE")
        print("="*40)

        score = evaluate_final_state(grid, p1_troops, p2_troops, initiative_winner)
        winner = "Player 1" if score == 1 else "Player 2"

        display_grid(grid)
        print(f"\nSimulating the final attacks...")

        print("\n--- FINAL RESULT ---")
        if winner == human_player:
            print(f"\033[92mVICTORY! You defeated the AI!\033[0m")
        else:
            print(f"\033[91mDEFEAT. The AI was triumphant. Better luck next time.\033[0m")

    except Exception as e:
        print(f"\nAn error occurred: {e}")

if __name__ == "__main__":
    run_human_vs_ai_game()

--- Aggression: Human vs. AI ---
